# Spike Langgraph Meta Feedback

In [1]:
%pip install -Uq langchain langgraph langchain_openai langchainhub langsmith ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# Commented out if set in .env file
# os.environ["OPENAI_API_KEY"] = 'lm-studio' # Set to 'lm-studio' for local LMStudio inference
# os.environ["LANGCHAIN_API_KEY"] = ''

# Set LangSmith tracing and project name reference variables
os.environ["LANGCHAIN_TRACING_V2"] = 'true' 
os.environ["LANGCHAIN_PROJECT"] = 'Meta Feedback'

In [3]:
import functools, operator, requests, os, json
from bs4 import BeautifulSoup

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict

/Users/leonvanbokhorst/repos/press-to-talk/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Initialize model
llm = ChatOpenAI(model="gpt-3.5-turbo")
#llm = ChatOpenAI(model="NA", base_url="http://192.168.50.116:8000/v1") # Use for local inference

In [5]:
# Internet Search Tool (TavilySearch)
from langchain_community.tools.tavily_search import TavilySearchResults
internet_search = TavilySearchResults(max_results=3)

# Process Content Tool (BS4)
@tool("process_content", return_direct=False)
def process_content(url: str) -> str:
    """Processes content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

# Tool set
tools = [internet_search, process_content]

In [6]:
# Helper function for creating agents
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools)

# Define agent nodes
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [7]:
# Create Agent Supervisor
members = ["feedback_coach", "feedback_analyst", "transcript_checker"]
system_prompt = ("""
    As a supervisor, your role is to oversee a dialogue between these workers: 
    {members}. 
    
    Based on the user's request, determine which of the workers should take the next action. 
    Each worker is responsible for answering a specific point of view and reporting back their findings and progress. 
    Once all tasks are complete, indicate with 'FINISH'."""
)

options = ["FINISH"] + members
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}] }},
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Given the conversation above, who should act next? We can only FINISH only after all workers gave you an advise? Select one of: {options}"),
]).partial(options=str(options), members=", ".join(members))

supervisor_chain = (prompt | llm.bind_functions(functions=[function_def], function_call="route") | JsonOutputFunctionsParser())

In [8]:
# Create Feedback Analyzer Agent
feedback_analyst = create_agent(
    llm,
    tools,
    """
	You are a very smart feedback analyst. 
	You can recognize given feedback, feed forward and feed up from written text. 
	Whenever asked, you provide a deep analysis of the quality of the feedback.""",
)

feedback_analyst_node = functools.partial(
    agent_node, agent=feedback_analyst, name="feedback_analyst"
)

In [9]:
# Create Feedback Coach Agent
feedback_coach = create_agent(
    llm,
    tools,
    """
    You are a compassionate feedback coach. 
    You can give growth oriented feedback, feed forward and feed up from written text. 
    Whenever asked, you provide feedback to enhance the users learning process, personal, and professional growth.""",
)

feedback_coach_node = functools.partial(agent_node, agent=feedback_coach, name="feedback_coach")

In [10]:
# Create Transcript Agent
transcript_checker = create_agent(
    llm,
    tools,
    """
    You are a professional transcript checker. 
    You know exactly what is, and what is not mentioned in the transscript. 
    Whenever asked, you provide only the straight facts from the transcript in bullet points.""",
)

transcript_checker_node = functools.partial(agent_node, agent=transcript_checker, name="transcript_checker")

In [11]:
# Define the Agent State
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

workflow = StateGraph(AgentState)
workflow.add_node("feedback_coach", feedback_coach_node)
workflow.add_node("feedback_analyst", feedback_analyst_node)
workflow.add_node("transcript_checker", transcript_checker_node)
workflow.add_node("supervisor", supervisor_chain)

In [12]:
# Define edges
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

# Compile the workflow graph
graph = workflow.compile()

In [15]:
# Run the graph
from IPython.display import display, Markdown


for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="""
                I want to know if I gave enough encouragement to the student in the conversation below.
                
                Here is the transcript:
                
                --- Transcript ---
                Student: I am struggling with my math homework.
                Teacher: What specifically are you struggling with?
                Student: I don't understand how to do long division.
                
                Teacher: Ok, let's go through an example together.
                Student: Ok.
                
                Teacher: What is 100 divided by 5?
                Student: 20.
                
                Teacher: Incorrect. What is 100 divided by 4?
                Student: 25.
                
                Teacher: Correct. So why are you asking me then? You know your shit :)
                
                But seriously, great job. No try to do it on your own for the next hour and let's see how far you can get.
                
                """
            )
        ],
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'feedback_coach'}}
----
{'feedback_coach': {'messages': [HumanMessage(content='It\'s great that you\'re seeking feedback on your encouragement approach with the student. Based on the conversation provided, here are some points to consider for enhancing your encouragement:\n\n1. **Positive Reinforcement**: You did provide positive reinforcement by acknowledging the student\'s correct answer. This is important for boosting confidence and motivation.\n\n2. **Constructive Feedback**: While it\'s good to acknowledge the correct answer, it\'s also essential to address mistakes constructively. Instead of saying "Incorrect," you could provide guidance on how to arrive at the correct answer.\n\n3. **Supportive Tone**: Your tone, especially the phrase "You know your shit :)" can be perceived in different ways by different students. It\'s crucial to maintain a professional and supportive tone throughout the interaction.\n\n4. **Encouragement for Independence**: Encouraging